<a href="https://colab.research.google.com/github/dimasrfaria/falvojr-santander-dev-week-2023/blob/master/santanderDevWeek_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FASE "E" -> EXTRACT**

In [21]:
# --------------------------------
# 1. Leitura do CSV de cadastro dos usuários com uso do pandas - Temos somente 3 usuários devido a limitação da API gratuita no Gemini que é a utilizada neste teste.
# --------------------------------

import pandas as pd

df = pd.read_csv('SDW2026.csv', sep=';')


# **FASE "T" -> TRANSFORMATION**

In [ ]:

# Exibe informações sobre os dados para avaliação da Extração realizada (tipos e uso de memória)
#df.info()

#print("\n" * 3)  # 3 linha em branco para melhor visualização

# --------------------------------
# Exibe todas as informações em formato de tabela para avaliação dos dados
# Usado desta forma, por saber que são poucos dados
# --------------------------------

print(df.to_string(index=False))

print("\n" * 3)  # 3 linha em branco para melhor visualização

# --------------------------------
# 2. Importação das bibliotecas necessárias para implementação:
# ✔ Timestamp para auditoria
# ✔ Integração real com API OpenIA para mensagens personalizadas por usuário
# ✔ Travalhar com ".env" para evitar vazamento da chave OpenIA
# --------------------------------

import os
from datetime import datetime
from dotenv import load_dotenv

# --------------------------------
# 3. Escolha da IA (Somente utilizo o gemini para esse teste, pois a OpenIA não ofereceu a API gratuíta, mais verificado que esta funcionando.
#    o erro que é gerado é somente devido a falta de crédito)
# --------------------------------
print("\nSelecione a IA para geração das mensagens:")
print("1) OpenAI")
print("2) Gemini")

opcao = input("Digite 1 ou 2: ").strip()

if opcao not in ("1", "2"):
    raise ValueError("Opção inválida.")

# --------------------------------------------------
# 4. Carregar variáveis de ambiente e inicializa a API selecionada (OpenIA ou Gemini)
# --------------------------------------------------
load_dotenv()

# --------------------------------
# 5. Inicialização do provedor
# --------------------------------
provider = None

if opcao == "1":
    from openai import OpenAI

    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
    if not OPENAI_API_KEY:
        raise RuntimeError("OPENAI_API_KEY não encontrada.")

    provider = "openai"
    client = OpenAI(api_key=OPENAI_API_KEY)


elif opcao == "2":
    from google import genai
    import time

    GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
    if not GEMINI_API_KEY:
        raise RuntimeError("GEMINI_API_KEY não encontrada.")

    provider = "gemini"
    #genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
    #model = genai.GenerativeModel("gemini-1.5-flash")
    client_gemini = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

# --------------------------------
# 6. Função ÚNICA de geração
# --------------------------------
def gerar_news(usuario):
    prompt = f"""
    Você é um especialista em marketing bancário.

    Crie uma mensagem curta, profissional e clara para o cliente abaixo,
    incentivando o uso de soluções financeiras do banco.
    Não use linguagem agressiva de vendas.

    Nome do cliente: {usuario['Nome']}

    Diretrizes:
    - Tom institucional e profissional
    - Linguagem clara e objetiva
    - Não mencionar taxas, valores, nome do banco ou produtos específicos
    - Não inventar informações pessoais
    - Máximo de 3 frases

    """

    try:
        if provider == "openai":
            resposta = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "Você é um especialista em marketing bancário com foco em compliance."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.6,
                max_tokens=120
            )
            return resposta.choices[0].message.content.strip()

        elif provider == "gemini":
            resposta = client_gemini.models.generate_content(
                model="gemini-2.0-flash",
                contents=prompt
            )
            return resposta.text.strip()

    except Exception as e:
        print(f"Erro {provider} para o usuário {usuario['UserID']}: {e}")
        return (
            "Temos novidades financeiras pensadas para apoiar seus objetivos "
            "com segurança e praticidade."
        )



# -----------------------------
# 7. Geração das mensagens (News)
# -----------------------------
news_geradas = []

data_geracao = datetime.now().strftime("%d-%m-%y %H:%M:%S")

for _, usuario in df.iterrows():
    news_geradas.append({
        "UserID": usuario["UserID"],
        "Nome": usuario["Nome"],
        "News": gerar_news(usuario),
        "DataGeracao": data_geracao

    })

    time.sleep(3)  # Pausa de 3 segundos entre cada cliente para não travar a API gratuita

df_news = pd.DataFrame(news_geradas)
print("\n--- Fase transformation Finalizado ---")


# **FASE "L" -> LOAD**

In [23]:
# -----------------------------
# 8. Escrita no CSV com histórico
# -----------------------------
arquivo_saida = "SDW2026_news.csv"
arquivo_existe = os.path.exists(arquivo_saida)

df_news.to_csv(
    arquivo_saida,
    sep=";",
    index=False,
    encoding="utf-8",
    mode="a",                 # append (mantém histórico)
    header=not arquivo_existe # escreve cabeçalho só se o arquivo não existir
)
print("\n--- Fase de Load Finalizado ---")


--- Fase de Load Finalizado ---


# **Alguns dos comandos utilizados para instação e validação dos recursos utilizados**

In [ ]:
#-----------------------------
# Intalação da biblioteca, para suporte ao uso do .env no python, afim de ocultar a chave da OpenIA
#-----------------------------

#pip install python-dotenv

In [24]:
#-----------------------------
#Teste de leitura para a variavel de ambiente ".env"
#-----------------------------

#from dotenv import load_dotenv
#load_dotenv()


True

In [ ]:
#-----------------------------
# validar se está funcionando a chave de ambiente .env
#-----------------------------

#import os
#print(os.getenv("GEMINI_API_KEY"))